In [1]:
import pandas as pd
import gmaps
import gmaps.geojson_geometries
from matplotlib.cm import magma
from matplotlib.colors import to_hex
import requests


# NEED TO SET UP API KEYS
# Let 'gkey' in config be the Google maps api key

from config import gkey

In [2]:
# First, we need to clean the horribly formatted initial dataset into something usable.

path = 'resources/centers-uncleaned.csv'
with open(path, 'r') as csv_file:
    output = csv_file.read()
output = output.split('\n')

all_name = []
all_city = []
all_state = []
all_zip = []

for i in range(len(output)):
    info = output[i].split(';')
    try:
        name = info[1]
        # This loop searches for any entry that looks like a city / state / zip combination; if one is found it's saved, otherwise we ignore that row, since we don't have any useful information about that center.
        for j in range(2, len(info) + 1):
            try:
                if(all(x in '1234567890' for x in info[j][-5:])):
                    area = info[j]
                    break
            except:
                pass
        
        city = area.split(',')[0]
        state = area.split(',')[1][1:].split()[0]
        zipcode = area.split(',')[1][1:].split()[1]
        all_name.append(name)
        all_city.append(city)
        all_state.append(state)
        all_zip.append(zipcode)
    except:
        pass

In [3]:
# Create a nicely formatted dataframe to work with

centers_df = pd.DataFrame({'Name' : all_name, 'City' : all_city, 'State' : all_state, 'Zip' : all_zip})

zips_df = pd.read_csv('resources/zips.csv', dtype = {'ZIP' : 'str'})
zips_df = zips_df.rename(columns = {'ZIP' : 'Zip', 'LAT' : 'Lat', 'LNG' : 'Lon'})
centers_df = centers_df.merge(zips_df, on = 'Zip', how = 'inner')

out_path = 'resources/centers.csv'
centers_df.to_csv(out_path)

In [4]:
centers_df.head()

,Name,City,State,Zip,Lat,Lon
0,Henry County Day Treatment,Abbeville,AL,36310,31.595148,-85.208852
1,SpectraCare,Abbeville,AL,36310,31.595148,-85.208852
2,Rehab Day Montezuma Complex,Andalusia,AL,36420,31.172401,-86.565077
3,South Central Alabama CMHC,Andalusia,AL,36420,31.172401,-86.565077
4,Grandview Behavioral Health Center,Anniston,AL,36201,33.649160,-85.879642


In [5]:
gmaps.configure(api_key = gkey)

In [10]:
locations = centers_df[['Lat', 'Lon']].drop_duplicates()
fig = gmaps.figure()
num_centers = centers_df.groupby(by = 'Zip').count()['Name']
heat_layer = gmaps.heatmap_layer(locations, weights = num_centers, max_intensity = 85, point_radius = 6)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
state_df = centers_df.loc[~centers_df['State'].isin(['GU', 'VI', 'AS'])].groupby(by = 'State')['Name'].count()
state_df.to_csv('resources/centers_bystate.csv')
state_df.head()

State
AK     80
AL    133
AR    204
AZ    305
CA    670
Name: Name, dtype: int64

In [8]:
# this block is to order the states in the weird order that is needed for the state_geojson in the next block, so that they can be colored properly.

url = 'https://s3-eu-west-1.amazonaws.com/jupyter-gmaps-examples/us-states.geo.json'
weird_order = requests.get(url).json()
states_order = []
for i in range(52):
    states_order.append(weird_order['features'][i]['properties']['NAME'])
    
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia' : 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico' : 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

state_order_abbr = []
for state in states_order:
    state_order_abbr.append(us_state_abbrev[state])

In [9]:
state_geojson = gmaps.geojson_geometries.load_geometry('us-states')
fig = gmaps.figure()
min_centers = int(min(state_df))
max_centers = int(max(state_df))
range_centers = max_centers - min_centers

def calculate_color(count):
    normalized = (int(count) - min_centers) / range_centers
    mpl_color = magma(normalized)
    gmaps_color = to_hex(mpl_color, keep_alpha = False)
    return gmaps_color

colors = []

for state in state_order_abbr:
    colors.append(calculate_color(state_df[state]))

centers_layer = gmaps.geojson_layer(state_geojson, fill_color = colors, stroke_color = colors, fill_opacity = 0.8)
fig.add_layer(centers_layer)
fig

Figure(layout=FigureLayout(height='420px'))